#**Cleaning columns name**

## **Table 1**

In [0]:
df = spark.sql("select * from workspace.bronze.crm_cust_info")
df.display()

In [0]:
r = [f.name for f in spark.catalog.listTables("workspace.bronze")]

In [0]:
r

In [0]:
df.dtypes

In [0]:
def rename_columns(df):
    for c in df.columns:
        if c == 'cst_gndr':
            df = df.withColumnRenamed(c, c.replace("cst_gndr","gender"))
        else:
            df = df.withColumnsRenamed({c: c.replace("cst_","")})

    return df

df = rename_columns(df)
df.display()

In [0]:
from pyspark.sql.functions import trim, when, col

def removing_spaces_columns(df):
    for c in df.columns:
        df = df.withColumn(c, trim(df[c]))
        if c == 'gender':
            df = df.withColumn(
                c,
                when(col(c) == "M", "Male")
                .when(col(c) == "F", "Female")
                .otherwise(col(c))
            )
        elif c == 'marital_status':
            df = df.withColumn(
                c,
                when(col(c) == "S", "Single")
                .when(col(c) == "M", "Married")
                .otherwise(col(c))
            )
    return df

df = removing_spaces_columns(df)

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import count, when, col
spark.sql("select count(*) as total from workspace.bronze.crm_cust_info")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).display()

In [0]:
#null values
df = df.fillna('Unknown')
df = df.fillna(0, subset=['id'])
df = df.fillna(0000-00-00, subset=['create_date'])
df.display()
df.write.mode("overwrite").saveAsTable("workspace.silver.slvr_customer_info")

In [0]:
df.dtypes

Table 2

In [0]:
df1 = spark.sql("select * from workspace.bronze.crm_prd_info")
df1.display()

In [0]:
from pyspark.sql.functions import count, when, col
df1.select([count(when(col(c).isNull(), c)).alias(c) for c in df1.columns]).display()

In [0]:
from pyspark.sql.functions import to_date, lit, when, col

df1 = df1.fillna('Unknown', subset=['prd_line'])
df1 = df1.fillna(0, subset=['prd_cost'])
df1 = df1.withColumn('prd_end_dt', when(col("prd_end_dt").isNull(), to_date(lit("1900-01-01")))
                     .otherwise(col('prd_end_dt')))
df1.display()

In [0]:
df1.write.mode("overwrite").saveAsTable("workspace.silver.slvr_product_info")


Table 3 

In [0]:
import pyspark.sql.functions as sf

t3 = spark.sql("select * from workspace.bronze.crm_sales_details")
t3.display()

In [0]:
t3 = t3.withColumn('sls_order_dt', sf.col('sls_order_dt').cast('string'))
t3 = t3.withColumn('sls_ship_dt', sf.col('sls_ship_dt').cast('string'))
t3 = t3.withColumn('sls_due_dt', sf.col('sls_due_dt').cast('string'))

In [0]:
t3 = t3.withColumn('sls_order_dt', sf.when(sf.length(sf.col('sls_order_dt')) < 8, None).otherwise(sf.col('sls_order_dt')))

In [0]:
t3 = t3.withColumn('sls_order_dt', sf.expr("try_to_date(sls_order_dt, 'yyyyMMdd')"))
t3 = t3.withColumn('sls_ship_dt', sf.expr("try_to_date(sls_ship_dt, 'yyyyMMdd')"))
t3 = t3.withColumn('sls_due_dt', sf.expr("try_to_date(sls_due_dt, 'yyyyMMdd')"))

In [0]:
t3.dtypes

In [0]:
t3.display()

In [0]:
t3.display()

In [0]:
# filling nulls in columns
t3 = t3.withColumn("sls_price", sf.when(sf.col("sls_price").isNull(), 0).otherwise(sf.col("sls_price")))


In [0]:
t3 = t3.withColumn('sls_sales', sf.when(sf.col('sls_sales').isNull(), 0).otherwise(sf.col('sls_sales')))


In [0]:
# There is no error in this line. It correctly writes t3 to the specified table in overwrite mode.
t3.write.mode("overwrite").saveAsTable("workspace.silver.slvr_sales_details")

Table 4


In [0]:
import pyspark.sql.functions as sf

t4 = spark.sql("select * from workspace.bronze.erp_cust")
t4.display()


In [0]:
t4.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in t4.columns]).display()

In [0]:
t4 = t4.withColumn("GEN", sf.trim(sf.col('GEN')))
t4 = t4.withColumn("GEN", sf.when(sf.col("GEN").isin(" ", "  ",""), "Unknown").otherwise(sf.col("GEN")))
t4 = t4.withColumn("GEN", sf.when(sf.col('GEN').isNull(), "Unknown").otherwise(sf.col("GEN")))
t4 = t4.withColumn("GEN", sf.when(sf.col("GEN") == "M", "Male")
                          .when(sf.col("GEN") == "F", "Female")
                          .otherwise(sf.col("GEN")))
t4 = t4.withColumnRenamed("GEN",'gender')

In [0]:
t4.groupBy("gender").count().display()


In [0]:
t4.write.mode("overwrite").saveAsTable("workspace.silver.slvr_erp_cust")

Table 5

In [0]:
t5 = spark.sql("select * from workspace.bronze.erp_category")

In [0]:
t5 = t5.withColumnRenamed("CAT",'category')
t5 = t5.withColumnRenamed("SUBCAT",'subcategory')
t5 = t5.withColumnRenamed("MAINTENANCE",'maintenance')

In [0]:
t5.display()

In [0]:
t5.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in t5.columns]).display()

In [0]:
t5.write.mode("overwrite").saveAsTable("workspace.silver.slvr_erp_category")

T6 Table

In [0]:
t6= spark.sql('select * from workspace.bronze.erp_loc')

In [0]:
t6.display()

In [0]:
t6.dtypes

In [0]:
t6 = t6.withColumnRenamed("CID", "CID").withColumnRenamed("CNTRY", "COUNTRY")

In [0]:
t6.display()

In [0]:
t6.groupBy('COUNTRY').count().display()

In [0]:
t6 = t6.withColumn('COUNTRY', sf.when(sf.col('COUNTRY') == 'DE', 'Germany').otherwise(sf.col('COUNTRY'))) \
       .withColumn('COUNTRY', sf.when((sf.col('COUNTRY') == 'United States') | (sf.col('COUNTRY') == 'US'), 'USA').otherwise(sf.col('COUNTRY'))) \
       .withColumn('COUNTRY', sf.trim(sf.col('COUNTRY'))) \
       .withColumn('COUNTRY', sf.when(sf.col('COUNTRY')=='',None).otherwise(sf.col('COUNTRY'))) \
       .withColumn('COUNTRY', sf.when(sf.col('COUNTRY').isNull(), 'Unknown').otherwise(sf.col('COUNTRY')))

In [0]:
t6.write.mode('overwrite').saveAsTable('workspace.silver.slvr_erp_locatiom')